In [1]:
import geopandas as gpd
import pandas as pd
import verde as vd
import numpy as np
from shapely import geometry


import matplotlib.pyplot as plt
%matplotlib widget

gdb='/home/ggrl/geodatabase/'

In [2]:
g_rparaim = pd.read_csv(gdb+'geof/recortado/g_rparaim')

In [3]:
rparaim = gpd.read_file(gdb+'geodatabase.gpkg',
                        driver='GPKG',
                        layer='l_100k')[gpd.read_file(gdb+'geodatabase.gpkg',
                            driver='GPKG',
                            layer='l_100k').MAPA == 'Carta geológica da folha Rio Paraim']

In [4]:
rparaim.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [5]:
rparaim.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
rparaim = rparaim.to_crs('EPSG:32723')

In [27]:
rparaim.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [29]:
g_rparaim.head()

,X,Y,MDT,KPERC,eU,eth,CTCOR,LONGITUDE,LATITUDE
0,500020.25,8786155.43,477.68,0.35,1.25,16.12,1182.85,-44.999815,-10.980692
1,500103.83,8786158.77,476.83,0.63,1.24,14.00,1143.89,-44.999050,-10.980662
2,500187.24,8786161.64,476.58,1.02,1.38,12.02,1165.74,-44.998286,-10.980636
3,500270.50,8786163.59,470.91,1.41,1.51,11.31,1257.68,-44.997524,-10.980618
4,500353.54,8786165.69,464.63,1.71,1.65,11.14,1357.00,-44.996764,-10.980599


In [37]:
chain_list = ['CTCOR', 'KPERC', 'MDT', 'eU', 'eth']
chain = ['CTCOR', 'KPERC', 'MDT', 'eU', 'eth']

coordinates = (g_rparaim.X.values, g_rparaim.Y.values)

for i in chain_list:
    print(i)

    chain = vd.Chain([
        ('trend', vd.Trend(degree=1)),
        ('reduce', vd.BlockReduce(np.median, spacing=1000)),
        ('spline', vd.Spline())
    ])

    chain.fit(coordinates, g_rparaim[i])
    
    grid = chain.grid(spacing=200, data_names=[i])
    cut_grid = vd.distance_mask(coordinates, maxdist=1000, grid= grid)

    geof = CTCOR_grid.to_dataframe()
    geof = geof.reset_index(inplace=True)
    break



CTCOR


In [43]:
cut_grid

<xarray.Dataset>
Dimensions:   (easting: 274, northing: 278)
Coordinates:
  * easting   (easting) float64 5e+05 5.002e+05 ... 5.545e+05 5.547e+05
  * northing  (northing) float64 8.784e+06 8.784e+06 ... 8.839e+06 8.839e+06
Data variables:
    CTCOR     (northing, easting) float64 1.66e+03 1.679e+03 ... 1.172e+03
Attributes:
    metadata:  Generated by Chain(steps=[('trend', Trend(degree=1)),\n       ...

In [8]:
coordinates = (g_rparaim.X.values, g_rparaim.Y.values)

chain_CTCOR = vd.Chain([
    ('trend',  vd.Trend(degree=1)),
    ('reduce', vd.BlockReduce(np.median, spacing=1000)),
    ('spline', vd.Spline()),
])

chain_CTCOR.fit(coordinates, g_rparaim.CTCOR)

grid_CTCOR = chain_CTCOR.grid(spacing=200, data_names=['CTCOR'])
CTCOR_grid = vd.distance_mask(coordinates, maxdist=1000, grid=grid_CTCOR)

geof = CTCOR_grid.to_dataframe()
geof.reset_index(inplace=True)

In [ ]:
coordinates = (g_rparaim.X.values, g_rparaim.Y.values)

chain_CTCOR = vd.Chain([
    ('trend',  vd.Trend(degree=1)),
    ('reduce', vd.BlockReduce(np.median, spacing=1000)),
    ('spline', vd.Spline()),
])

chain_CTCOR.fit(coordinates, g_rparaim.KPERC)

grid_CTCOR = chain_CTCOR.grid(spacing=200, data_names=['CTCOR'])
CTCOR_grid = vd.distance_mask(coordinates, maxdist=1000, grid=grid_CTCOR)


CTCOR_geof = CTCOR_grid.to_dataframe()
CTCOR_geof = CTCOR_geof.reset_index(inplace=True)

In [28]:
rparaim.plot(figsize=(9,9))
plt.scatter(geof['easting'], geof['northing'], c= geof['CTCOR'],cmap='magma', s=0.5)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(497264.37192502536, 557448.1895744522, 8781208.59827951, 8842071.396711774)

In [24]:
'''
bounds =  rparaim.unary_union.bounds

reg = (bounds[0],bounds[2],
          bounds[1],bounds[3])
'''
region = vd.get_region(coordinates)
grid_coords = vd.grid_coordinates(region, spacing = 200,pixel_register=True)

In [26]:
grid_coords.plot()

AttributeError: 'tuple' object has no attribute 'plot'

In [60]:
geof['geometry'] = [geometry.Point(x,y) for x, y in zip(geof['easting'], geof['northing'])]

In [61]:
gdf = gpd.GeoDataFrame(geof)

In [62]:
gdf = gdf.set_crs('EPSG:32723')

In [63]:
gdf.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [64]:
rparaim.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [65]:
base = rparaim.plot('SIGLA')
gdf.plot(ax=base,markersize=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [78]:
rparaim.plot('SIGLA')
plt.scatter(grid_coords[0],grid_coords[1],cmap='black',s=2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
len(grid_coords[0])

277

In [92]:
pd.DataFrame(np.column_stack(grid_coords[0]))

,0,1,2,3,4,5,6,7,8,9,...,267,268,269,270,271,272,273,274,275,276
0,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198,...,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198,500105.462198
1,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593,...,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593,500305.786593
2,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989,...,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989,500506.110989
3,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385,...,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385,500706.435385
4,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780,...,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780,500906.759780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220,...,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220,553792.400220
269,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615,...,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615,553992.724615
270,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011,...,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011,554193.049011
271,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407,...,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407,554393.373407


In [47]:
gdf['closest_unid'] = \
    rparaim['geometry'].apply(lambda x: rparaim['SIGLA'].iloc[rparaim.distance(x).idxmin()])

IndexError: single positional indexer is out-of-bounds